In [17]:
import pandas as pd
import os
from IPython.display import display
import sys
# from tabulate import tabulate

pd.set_option('mode.chained_assignment', None)

In [18]:
initial_amount = 500
months_to_distribute_initial_amount = 5
term_months = 10
term_holidays = 2
credit_rate_yearly = 0.12
total_time_in_months = 120
total_time_in_months_test = total_time_in_months#total_time_in_months#24

credit_rate_monthly = credit_rate_yearly/12
initial_payment = initial_amount/months_to_distribute_initial_amount

In [19]:
cl_dct = {}
col_names_basic = ["Term","Amount_left"]
col_names_clients = ["Paid","Return_principal","Return_interest"]

cl_data_ini = []
for i in range(total_time_in_months_test+1):
  cl_data_ini.append(("m%03d"%(i+1),None))
df = pd.DataFrame(cl_data_ini,columns=col_names_basic)

amount_left = initial_amount
amount_available = [amount_left]
# print("Available amount in the beginning of month:")
# print("month_%02d: %0.2f$"%(1,amount_left))


for clt in range(total_time_in_months_test):#-term_months):
  client_number = clt+1
  
  cl_data_ini = []
  if client_number <= total_time_in_months_test-term_months+1:
    for i in range(total_time_in_months_test+1):
      if i+1 < client_number:
        cl_data_ini.append(("m%03d"%(i+1),None,0.0,0.0,0.0))
      elif i+1 == client_number:
        payment = initial_payment if initial_payment <= amount_left else amount_left
        amount_left -= payment
        return_principal = payment/(term_months-term_holidays)
        principal_left = payment
        cl_data_ini.append(("m%03d"%(i+1),amount_left,payment,0.0,0.0))

      elif i < client_number + term_holidays:
        cl_data_ini.append(("m%03d"%(i+1),amount_left,payment,0.0,0.0))
      elif i < term_months + client_number:
        cl_data_ini.append(("m%03d"%(i+1),None,0.0,return_principal,principal_left*credit_rate_monthly))
        principal_left -= 0#return_principal
      else:
        cl_data_ini.append(("m%03d"%(i+1),None,0.0,0.0,0.0))

    df0 = pd.DataFrame(cl_data_ini,columns=col_names_basic + list(map(lambda x:x+"_%03d"%client_number,col_names_clients)))  
    df0[col_names_basic[-1]][:client_number-1] = df[col_names_basic[-1]][:client_number-1]
    df = df.drop(col_names_basic[-1],axis=1,).merge(df0,on=col_names_basic[0],sort=True)

    cols = list(df)
    cols.insert(1,cols.pop(cols.index(col_names_basic[-1])))
    df = df[cols]
    
    amount_left += df.loc[client_number,[col for col in df if col.startswith(col_names_clients[1])]].sum()
  else:
    df[col_names_basic[-1]][client_number-1] = amount_left  
    amount_left += df.loc[client_number,[col for col in df if col.startswith(col_names_clients[1])]].sum()
    
    df[col_names_basic[-1]][client_number] = amount_left
    
  amount_available.append(amount_left)
#   print("month_%02d: %0.2f$"%(client_number+1,amount_left))
  
# display(df)
df["Total_return_interest"] = df[[col for col in df if col.startswith(col_names_clients[2])]].sum(axis=1)
df[["Total_cumul_return_interest"]] = df[["Total_return_interest"]].cumsum()
df["Amount_available"] = amount_available

df_result = df[["Term"]+[col for col in df if col.startswith(("Amount","Total_"))]]
df_result = df_result[['Term','Amount_available', 'Amount_left', 'Total_return_interest', 'Total_cumul_return_interest']]
df_result[df_result.columns[1:]] = df_result[df_result.columns[1:]].applymap('{:,.02f}'.format)

display(df_result)
# display(df)

,Term,Amount_available,Amount_left,Total_return_interest,Total_cumul_return_interest
0,m001,500.00,400.00,0.00,0.00
1,m002,400.00,300.00,0.00,0.00
2,m003,300.00,200.00,0.00,0.00
3,m004,212.50,112.50,1.00,1.00
4,m005,137.50,37.50,2.00,3.00
5,m006,75.00,0.00,3.00,6.00
6,m007,50.00,0.00,4.00,10.00
7,m008,62.50,0.00,5.00,15.00
8,m009,71.88,0.00,5.75,20.75
9,m010,78.12,0.00,6.25,27.00
